<a href="https://colab.research.google.com/github/coldsober-irene/hackathon1/blob/main/Copy_of_hackathon1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  -U easynmt

In [ ]:
!pip install uvicorn fastapi

In [ ]:
!pip install fastapi pyngrok uvicorn nest-asyncio

In [25]:
# config the account
!ngrok authtoken 2HJPvK5e4VxcduBjO5XSc0LqqCR_5zV3b5gBcKuNMUnxdpfuP

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [28]:
# REQUIRED IMPORTS
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import json
import numpy as np
from easynmt import EasyNMT
import json # to access languages codes
from fastapi import FastAPI, Response
import nest_asyncio
from pyngrok import ngrok
import uvicorn

In [3]:
# change the default limit of pandas columns to display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% **WEBSCRAPING SECTION** %%%%%%%%%%%%%%%%%%%%%

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# DATA MINING
class dataMining:
  def __init__(self):
    pass
    
  # SOUP
  def Soup(self, url):
    return bs(requests.get(url).content, "html.parser")
  
  def get_text(self):
    soup = self.Soup(url = 'https://www.jobinrwanda.com/jobs/featured')
    parent = "https://www.jobinrwanda.com"
    co_links = {}
    app_link = []
    full_text = [[(p.get_text(), self.Soup(j_link).find("a", class_ = "btn btn-sm btn-success")) for p in dv.find_all('p', class_ = 'text-align-justify')] for j_link in [job_link for job_link in [parent+link['href'] if '/job/' in link['href'] else co_links.setdefault(link.text, parent+link['href']) 
    for div in soup.find_all('div', class_ = 'card-body p-2')[4] for link in div.find_all('a')] if '/job/' in job_link]
    for dv in self.Soup(j_link).find_all('div', class_ = "clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item")
    ]
    return (full_text, app_link)

  def get_links(self):
    # GET SOUP OBJECT
    # soup = self.Soup(url = "https://www.jobinrwanda.com/jobs/all")
    soup = self.Soup(url = 'https://www.jobinrwanda.com/jobs/featured')
    # GET ALL JOBS LINKS
    company_infoLink = []
    
    all_jobsLinks = ["https://www.jobinrwanda.com"+link['href'] if "job" in link['href'] 
                     else company_infoLink.append(link.text) 
                     for div in soup.find_all("div", class_= "card-body p-2") 
                     for link in div.find_all("a")
                     ]
    all_jobsLinks = [link for link in all_jobsLinks if link]
    return (all_jobsLinks, company_infoLink)

  def get_jobInfo(self):
    jobLinks = self.get_links()[0]
    info = {}
    # REFINE THE INFO INTO A DICTIONARY
    def refine_info(ls):
      refined_info = {}
      infoZeroPattern = re.compile("\d+.*")
      views = infoZeroPattern.search(ls[0]).group()
      refined_info["views"] = views
      # OTHER DETAILS EXCEPT THE ONE ON POSITION 0
      otherInfoPattern = re.compile(":")
      for detail in ls[1:]:
        if detail.lower() == "apply":
          pass
        else:
          try:
            detailed = otherInfoPattern.split(detail)
            refined_info[detailed[0]] = detailed[1]
          except IndexError:
            pass
      return refined_info
       
    for index,url in enumerate(jobLinks):
      soup = self.Soup(url)
      pattern = re.compile("\s{2,}")
      job_info = [pattern.sub(" ",li.text.strip().replace("\n", "")) for ul in soup.find_all('ul', class_ = "list-group list-group-flush") 
      for li in ul.find_all('li')]
      # DICTIONANRY OF INFO
      info[index] = refine_info(job_info[:9])
    return info

  def get_description(self):
    jobLinks = self.get_links()[0]
    all_text= {}
    for index,link in enumerate(jobLinks):
      soup = self.Soup(link)
      # JOB TITLE
      job_title = soup.find('span', class_ = "field field--name-title field--type-string field--label-hidden")
      
      tags_content = soup.find_all('div', class_= "clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item")
      
      for div in tags_content:
        temp = []
        try:
          # GET APPLICATION LINK
          app_link = soup.find("a", class_ = "btn btn-sm btn-success")['href']
          if app_link.startswith("/"):
            app_link = "https://www.jobinrwanda.com"+app_link
          temp.append(app_link)
          
        except TypeError:
          temp.append(np.nan)
        for tag in div.children:
          try:
            temp.append(tag.get_text())
          except (AttributeError, TypeError):
            pass
        temp.insert(0, job_title.text)
        all_text[index] = temp
    return all_text

In [6]:
# MERGE JOB DESCRIPTION WITH THE JOB INFO
def final_textDemo():
  global info
  # CLASS OBJECT CREATION
  jobInRwanda = dataMining()
  job_text = jobInRwanda.get_description()
  info = jobInRwanda.get_jobInfo()

  final_demo = []
  for index in list(info.keys()):
    try:
      info_value = list(info[index].values())
      # ADD COMPANY HIRING
      co_info = jobInRwanda.get_links()[1]
      info_value.append(co_info[index])
      # INCLUDE JOB TITLE
      info_value.append(job_text[index][0])
      # INCLUDE APPLY LINK
      info_value.append(job_text[index][1])
      # JOIN PARAGRAPHS 
      t = '\n'.join(job_text[index][2:])
      g = re.compile("\n|\\xa0").sub("", t)
      info_value.append(g)
      final_demo.append(info_value)
    except KeyError:
      pass
  return final_demo
# FUNCTION CALL
final_content = final_textDemo()

In [7]:
# FORM DATAFRAME
titles = list(info[0].keys()) + ["Company Hiring",'Job Title', "Application link", "Job Description"]
len(final_content[0])
# , titles
df = pd.DataFrame(data = final_content, columns = titles)


In [8]:
df.tail(30)

,views,Location,Sector,Education level,Desired experience,Contract type,Deadline,Number of positions,Company Hiring,Job Title,Application link,Job Description
149,1231 times,"Kigali , RW",Administration,Bachelor,No experience,Part-time,"Friday, 18/11/2022 23",1,Embassy of the Federal Republic of Germany,"Assistant in the Press Department, Administrat...",https://www.jobinrwanda.com/form/default-job-a...,Summary:treatment of visa applicationspreparat...
150,371 times,"Kigali , RW",Other,Not specified,Not specified,Tender,"Friday, 11/11/2022 23",1,GIZ Rwanda,Expression of Interest (EOI)-Office Flooring W...,NaN,Expression of Interest (EOI)Office Flooring Wo...
151,407 times,", RW",Other,Not specified,Not specified,Tender,"Friday, 11/11/2022 23",1,GIZ Rwanda,Expression of Interest (EOI)-Office Partitioni...,NaN,Expression of Interest (EOI)Office Partitionin...
152,139 times,", RW",Other,,Tender,"Monday, 14/11/2022 17",1,Voluntary Service Overseas (VSO),Provision of Fuel (Petrol And Diesel) to Volun...,NaN,ADVERT TEMPLATE - EXPRESSION OF INTEREST (EOI)...,None
153,525 times,", RW",Other,Not specified,Not specified,Tender,"Saturday, 19/11/2022 10",1,Muganga SACCO,"Development of Mobile Banking Application, Imp...",NaN,INVITATION TO TENDERNo 009/T.MSC/2022/MSACCOMU...
154,1761 times,"Buhinga , RW","Agriculture, Agronomy, Demography and data an...",,Full-time,"Saturday, 19/11/2022 23",1,One Acre Fund,Rwanda Warehouse Manager,https://grnh.se/215948fc1us,"About One Acre FundFounded in 2006, One Acre F...",None
155,106 times,", RW",Other,Not specified,Not specified,Tender,"Sunday, 04/12/2022 11",1,ActionAid Rwanda (AAR),Request for Expression of Interest(EOI) for Pr...,NaN,REQUEST FOR EXPRESSION OF INTEREST FOR PRE‐QUA...
156,235 times,", RW",Other,Not specified,Not specified,Tender,"Sunday, 04/12/2022 11",1,ActionAid Rwanda (AAR),Request for Expression of Interest(EOI) for Pr...,NaN,REQUEST FOR EXPRESSION OF INTEREST FOR PRE‐QUA...
157,673 times,"Kigali , RW",Other,Bachelor,Mid career (3 to 5 years of experience),Full-time,"Monday, 21/11/2022 23",1,C Quest Capital,CSAT Officer – Rwanda,https://cquestcapital.bamboohr.com/jobs/view.p...,Job DescriptionCSAT Officer – RwandaCSAT Kigal...
158,589 times,", RW",Other,Not specified,Not specified,Contract,"Friday, 11/11/2022 17",1,RTI International,Request for Quote/Proposal (RFQ/RFP)- Pre-Qual...,NaN,Request for Quote/Proposal (RFQ/RFP)Commodity/...


# GET IT & COMPUTER RELATED JOBS, INTERNERSHIPS AND CONSULTANCY

In [9]:
# prepare kewords IT & COMPUTER jobs
IT_keywords = """
                #informationtechnology #technology #it #cybersecurity #tech #computerscience #programming #business #coding #innovation #software #python #information #computer #informationsecurity #security #technologynews #java #networking #hacking #programmer #linux #technologyrocks #coder #technologythesedays #cloudcomputing #education #engineering #itservices #newtechnology
              """.split('#')[1:-2]
# print(IT_keywords)
IT_keywords[2] = IT_keywords[2].upper()
keywords = re.sub('\s',"","|".join(IT_keywords))

In [ ]:
keywords = {'Job Description':keywords, 'Sector':'Computer|IT'}
df2 = df.copy()
df2['Is-IT-Job'] = False
df2['keywords found'] = None
def isIT_Job(targetCol = "Job Description"):
  pattern = re.compile(f"{keywords[targetCol]}")
  for index in range(len(df2)):
    try:
      if len(pattern.findall(df2.iloc[index][targetCol])) >= 5 and targetCol == "Job Description":
        df2['Is-IT-Job'][index] = True
        df2['keywords found'][index] = set(pattern.findall(df2.iloc[index][targetCol]))
      elif pattern.findall(df2.iloc[index][targetCol]) and targetCol == "Sector":
        df2['Is-IT-Job'][index] = True
        df2['keywords found'][index] = set(pattern.findall(df2.iloc[index][targetCol]))
    except TypeError:
      pass
isIT_Job()

In [20]:
df2[df2['Is-IT-Job'] == True]

,views,Location,Sector,Education level,Desired experience,Contract type,Deadline,Number of positions,Company Hiring,Job Title,Application link,Job Description,Is-IT-Job,keywords found
0,86 times,", RW",Other,Not specified,Not specified,Contract,"Friday, 25/11/2022 10",1,Development Bank of Rwanda (BRD),Hiring a Firm for BRD Maintenance Assessment,NaN,RE-ADVERTISEMENT OFTENDER : Nº: 022/10/2022/BR...,True,"{tech, IT}"
3,3002 times,"Kigali City , RW",Marketing and sales,High school,Entry level (1 to 3 years of experience),Full-time,"Friday, 11/11/2022 15",2,TIC-TAC-TOE,Customer Service Officer,https://www.jobinrwanda.com/form/default-job-a...,Job Description & Responsibilities:Assist in d...,True,"{computer, information, technology, tech, busi..."
4,2020 times,", RW","Administration, Business, Management, Other",Bachelor,Senior (5+ years of experience),Full-time,"Monday, 14/11/2022 23",1,GardaWorld,Operations Manager,https://www.jobinrwanda.com/form/default-job-a...,Job Description– Operations ManagerPosition/Em...,True,"{computer, security}"
5,608 times,", RW","Economics, Other, Social sciences",Master,Senior (5+ years of experience),Contract,"Tuesday, 15/11/2022 17",1,UN Women Rwanda,Terms of Reference for Production Of the Secon...,NaN,TERMS OF REFERENCE FOR PRODUCTION OF THE SECON...,True,"{tech, education, information, business}"
6,1973 times,", RW",Other,Not specified,Not specified,Other,"Monday, 14/11/2022 23",1,The leadership of Innovative Isiine Training C...,Call for Application for Professional Training...,https://docs.google.com/forms/d/e/1FAIpQLSfadD...,Call for application for Professional Training...,True,"{IT, business}"
7,3384 times,"Kigali , RW",Other,Bachelor,Senior (5+ years of experience),Full-time,"Friday, 11/11/2022 23",1,U.S. PEACE CORPS RWANDA,Finance Specialist (FS),https://www.jobinrwanda.com/form/default-job-a...,Vacancy Announcement:Finance Specialist (FS)BA...,True,"{computer, information, coding, IT, tech}"
8,3616 times,"Malawi, Uganda, Zambia, Rwanda , RW",Other,Not specified,Not specified,Full-time,"Sunday, 27/11/2022 23",1,Yellow,Account Manager- Credit,https://www.yellow.africa/eastafricaapply,Position:Account Manager- CreditLocations:Mala...,True,"{technology, tech, business}"
10,3 times,", RW",Other,Not specified,Not specified,Tender,"Friday, 18/11/2022 17",1,Rwanda Institute for Conservation Agriculture ...,Expression of Interest (EoI) for New Construct...,NaN,Call for Expression of Interest (EoI) for New ...,True,"{information, IT, tech}"
11,6 times,", RW","Administration, Business, Management, Other, ...",Bachelor,Senior (5+ years of experience),Contract,"Friday, 25/11/2022 23",1,GIZ Rwanda,Expression of Interest (EOI) Event management ...,NaN,Expression of Interest (EOI) Event management ...,True,"{information, security, IT, education, tech, n..."
13,59 times,", RW",Other,Not specified,Not specified,Tender,"Friday, 18/11/2022 12",1,Cleaner Production and Climate Innovation Cent...,Supply of Laptop Computers,NaN,TENDER NOTICE(TN)/INVITATION FOR BIDTender Not...,True,"{computer, IT, security}"


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% **TRANSLATOR SECTION** %%%%%%%%%%%%%%%%%%%%%

In [26]:
# try api 
app = FastAPI(title = 'display dataframe')
@app.get("/itjobs")
def show_it_jobs():
  return Response(df2.to_json(orient = "index"), media_type = "application/json")

In [29]:
ngrok_tunnel = ngrok.connect(8000)
print("[REST API STARTED]")
print("[YOUR PUCLIC API URL]: ", ngrok_tunnel.public_url)
# 
nest_asyncio.apply()
uvicorn.run(app, port = 8000)

INFO:     Started server process [78]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


[REST API STARTED]
[YOUR PUCLIC API URL]:  http://b707-34-143-176-189.ngrok.io
INFO:     102.22.164.159:0 - "GET /itjobs HTTP/1.1" 200 OK
INFO:     102.22.164.159:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [78]


In [ ]:
class translator:
  def __init__(self):
    # INITIALIZE THE MODEL FOR TRANSLATE
    self.model = EasyNMT("opus-mt")
  
  def supportedLangs(self, all_langs, supported):
    with open(all_langs) as f_json:
      content = f_json.read()
    langs = json.loads(content)
    print("langs: ", langs)

    with open(supported) as f2_json:
          content = f2_json.read()
    supplangs = json.loads(content)
    supp_codes = set(list(langs.values())).intersection(set(supplangs))
    supp_keyValue = {}
    for key in langs.keys():
      if langs[key] in supp_codes:
        supp_keyValue[key] = langs[key]
    return supp_keyValue
    

  def translate(self, sentences, targetLang, source = 'en', single = True):
    fail_message = "[FAIL]: destination lang is not a suported language by the model"
    success = 0
    if single:
      try:
        translation = self.model.translate(sentences, source_lang = source, target_lang = targetLang)
        return translation
      except:
        print(fail_message)
    else:
      translated = []
      for target in targetLang:
        try:
          translation = self.model.translate(sentences, source_lang = source, target_lang = target)
          translated.append(translation)
          success += 1
        except:
          print(fail_message)
      return translated, success


nest_asyncio : helps to make fastapi asynchronous

In [ ]:
# draft for consultancy
link = "https://www.jobinrwanda.com/jobs/consultancy"
soups = []
tags_contents = [[(child,soups.append(soup)) for child in final_div.children] for soup in [bs(requests.get(link).content, "html.parser") for link in ["https://jobinrwanda.com"+a['href'] 
        for div in soup.find_all('div', class_ = "card-body p-2") for a in div.find_all('a') if "job" in a['href']]]
         for final_div in soup.find_all('div', class_ = "clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item")
         ]



In [ ]:
c = "https://www.umucyo.gov.rw/eb/bav/selectListAdvertisingListForGU.do"
s = bs(requests.get(c).content, 'html.parser')
# print(s.prettify())

In [ ]:
table = s.find('table', class_ = "article_table mb10")

In [ ]:
titles = table.thead.text
heads = [re.compile('[\r|\n|\t]+').sub('#', head.text.strip()) for head in table.find_all('thead')]
heads = heads[0].split('#')
heads

In [ ]:
rows = table.find_all('tr')

In [ ]:
full = []
for row in rows:
  temp = []
  for detail in row.find_all('td'):
    temp.append(detail.text.strip())
  full.append(temp[1:])
  # print("*"*50)

In [ ]:
umucyo = pd.DataFrame(full, columns = heads)

In [ ]:
umucyo